### Graph based line merging

In [ ]:
import linegroupping as lg

In [ ]:
in_line = r"I:\Temp\centerline.shp"
out_line = r"I:\Temp\centerline_grouped.shp"

in_poly = r"I:\Temp\footprint_ground.shp"
out_poly = r"I:\Temp\footprint_ground_clean.shp"

# in_line = r"I:\Temp\centerline_S.shp"
# out_line = r"I:\Temp\centerline_grouped_S.shp"

# in_poly = r"I:\Temp\footprint_ground_S.shp"
# out_poly = r"I:\Temp\footprint_ground_clean_S.shp"

In [ ]:
lg_obj = lg.LineGroupping(in_line, in_poly)
lg_obj.run()
lg_obj.save_file(out_line, out_poly)

1. save original lines with group attributes for checkinng and editing
2. dissolve and generate footprint

### clip polygon and line
**line_merge and own merge tool runs after line clipped**